Init lib and create Repository to fetch the dataset

In [ ]:
import pandas as pd
from pandas_bokeh import output_notebook
pd.set_option('plotting.backend', 'pandas_bokeh')
output_notebook()

from repository.dataset import LocalDatasetRepository
datasetPath = './dataset.json'
datasetRepository = LocalDatasetRepository(datasetPath)

#from repository.dataset import RemoteDatasetRepository
#datasetURI = 'https://amazon-braket-anoir.s3.amazonaws.com/dataset.json'
#datasetRepository = RemoteDatasetRepository(datasetURI)

Create Read service and get dataset data

In [ ]:
from service.read_dataset import ReadDataset

readDatasetService = ReadDataset(datasetRepository)
datasetInMemory = readDatasetService.getAllData()

Create Export UnitOfWork

In [ ]:
from repository.export_result import LocalExportResultRepository, LocalSaveFactory, ExportResultUnitOfWork
from model.export_result import ResultUnit, ResultType
from service.export_file import ExportResultService
import os

file_dir = os.path.join(os.getcwd(), 'export')
if not os.path.isdir(file_dir):
    os.mkdir(file_dir)

exportLocalSaveMethods = LocalSaveFactory()
exportRepository = LocalExportResultRepository(file_dir, exportLocalSaveMethods)
exportService = ExportResultService(ExportResultUnitOfWork(exportRepository))

Init Data extractor service

In [ ]:
from service.extract_dataset import ExtractDataFromDataset
extractDataService = ExtractDataFromDataset(datasetInMemory)

Display crime distribution plot

In [ ]:
crimeDatesDF = extractDataService.getAllCrimesCountByDateDataframe().reset_index(name='Crimes')
crimeDatesFigure = crimeDatesDF.plot.line(
    title="Time series of the crimes",
    figsize=(800, 450),
    x="occur_date",
    color="mediumturquoise",
    xlabel="Date",
    ylabel="Crimes",
    toolbar_location=None,
    zooming=False,
    panning=False,
    legend=None,
    rangetool=True
)
exportService.addElementToExport(ResultUnit(crimeDatesFigure, "crime_dates.html", ResultType.BOKEH_PLOT))

Crimes frequency per type

In [ ]:
crimeTypeDF = extractDataService.getAllCrimesCountByTypeDataframe().reset_index(name='Frequency')
crimeTypeFigure = crimeTypeDF.plot_bokeh(
    title="Crimes frequency per type",
    figsize=(800, 450),
    kind="barh",
    color="darkslateblue",
    x="crime_type",
    xlabel="Frequency",
    ylabel="Crime Type",
    toolbar_location=None,
    zooming=False,
    panning=False,
    legend=None
)
exportService.addElementToExport(ResultUnit(crimeTypeFigure, "crime_type.html", ResultType.BOKEH_PLOT))

Display crime distribution by day

In [ ]:
crimeByDayDF = extractDataService.getAllCrimesCountByDayDataframe().reset_index(name='Frequency')
crimeByDayFigure = crimeByDayDF.plot_bokeh(
    title="Crimes frequency per day",
    figsize=(800, 450),
    kind="bar",
    color="indianred",
    x="avg_day",
    ylabel="Frequency",
    xlabel="Day",
    toolbar_location=None,
    zooming=False,
    panning=False,
    legend=None
)
exportService.addElementToExport(ResultUnit(crimeByDayFigure, "crime_day.html", ResultType.BOKEH_PLOT))

Display Type crime distribution by day

In [ ]:
dayAndFelonyDF = extractDataService.getAllCrimesCountGroupByDayAndFelony()#x = df.sort_values('avg_day').value_counts(subset=['avg_day', 'crime_type'])
dayAndFelonyFigure = dayAndFelonyDF.plot_bokeh(
    title="Crimes frequency per day",
    figsize=(800, 450),
    kind="bar",
    ylabel="Frequency",
    xlabel="Day",
    toolbar_location=None,
    zooming=True,
    panning=False,
    stacked=True,
    legend=None
)
exportService.addElementToExport(ResultUnit(dayAndFelonyFigure, "day_and_felony.html", ResultType.BOKEH_PLOT))

Top 50 Neighborhood with most crimes frequencies

In [ ]:
neighborhoodMostCriminalDF = extractDataService.getAllTop50NeighborhoodWithMostCrimesCountDataframe()
neighborhoodMostCriminalFigure = neighborhoodMostCriminalDF.plot_bokeh(
    title="Top 50 Neighborhood with most crimes frequencies",
    figsize=(800, 450),
    kind="barh",
    color="indianred",
    x="neighborhood",
    ylabel="Neighborhood",
    xlabel="Frequency",
    toolbar_location=None,
    zooming=False,
    panning=False,
    legend=None
)
exportService.addElementToExport(ResultUnit(neighborhoodMostCriminalFigure, "neighborhood_most_criminal.html", ResultType.BOKEH_PLOT))

Top 50 Neighborhood with less crimes frequencies

In [ ]:
neighborhoodLessCriminalDF = extractDataService.getAllTop50NeighborhoodWithLessCrimesCountDataframe()
neighborhoodLessCriminalFigure = neighborhoodLessCriminalDF.plot_bokeh(
    title="Top 50 Neighborhood with less crimes frequencies",
    figsize=(800, 450),
    kind="barh",
    color="indianred",
    x="neighborhood",
    ylabel="Neighborhood",
    xlabel="Frequency",
    toolbar_location=None,
    zooming=False,
    panning=False,
    legend=None
)

exportService.addElementToExport(ResultUnit(neighborhoodLessCriminalFigure, "neighborhood_less_criminal.html", ResultType.BOKEH_PLOT))

Init GIS Services and all repository

In [ ]:
from arcgis.gis import GIS
from service.arc_gis import ArcGISService, ArcGISPointStyle

gisRepository = GIS()
gisService = ArcGISService(gisRepository, use_cache=True)

Retrieve data by Homicide and display in map

In [ ]:
homicideDF = extractDataService.getAllCrimeByFelony("HOMICIDE")
homicideMap = gisService.getCrimesMapByFelonyType(homicideDF, ArcGISPointStyle.RED_POINT)
exportService.addElementToExport(ResultUnit(homicideMap, "homicide_map.html", ResultType.ARC_GIS_MAP))
homicideMap

Retrieve data by Homicide and display in map

In [ ]:
assaultDF = extractDataService.getAllCrimeByFelony("AGG ASSAULT")
assaultMap = gisService.getCrimesMapByFelonyType(assaultDF, ArcGISPointStyle.PURPLE_POINT)
exportService.addElementToExport(ResultUnit(assaultMap, "assault_map.html", ResultType.ARC_GIS_MAP))
assaultMap

Retrieve data by Homicide and display in map

In [ ]:
robberyResidenceDF = extractDataService.getAllCrimeByFelony("ROBBERY-RESIDENCE")
robberyResidenceMap = gisService.getCrimesMapByFelonyType(robberyResidenceDF, ArcGISPointStyle.GREEN_POINT)
exportService.addElementToExport(ResultUnit(assaultMap, "robbery_residence_map.html", ResultType.ARC_GIS_MAP))
robberyResidenceMap

Save all the Maps and plots and reset bokeh output

In [ ]:
exportService.completeExportProcess()

from pandas_bokeh import output_notebook
output_notebook()